IMMUNE SYSTEM ODES

patient constant H_i.

Parameters:
  Effector: a_E, b_EA, a_EHi, d_EP, d_EC (might need to add heavy side fucntion for chemo) (i think i can get away with just a_EHi, might not need a_E) 
  Helper:   a_H, b_HA, d_HP, d_HC, sigma_H, [b_HE=0]
  TIL:      a_I, b_IA, d_IP, d_IC
  Exhaustion: p_0, p_A, p_C, p_H, delta_P, rho_blk

RHS:
  dE = a_E*E + b_EA*E*A + a_EHi*E*H_i - d_EP*E*P - d_EC*E*C 
  dH = a_H*H + b_HA*H*A - d_HP*H*P - d_HC*H*C + b_HE*H*E
  dI = a_I*I + b_IA*I*A - d_IP*I*P - d_IC*I*C + u_I(t)
  dP = p_H*H_i + p_A*A + p_C*C - delta_P*P 

  Breakdown: Effector cell ode
  a_E*E - net growth rate (proliferation-death) of baseline CD8+ cells
  b_EA - added stimualtion of CD8+ cells due to tumor presence
  a_EHi - Flow of CD8+ cells depended upon patient immune system (unique to patient)
  d_EP - blockage of cd8+ cells due to PD-L1 cells
  d_EC - death of CD8+ cells due to chemo therapy

  Breakdown: CD4+ (Helper Cells)
  a_H - baseline flow of cd4+ cells (proliferation-death rate)
  b_HA - added flow of helper cells due to tumor cell presence
  d_HP - blockage of CD4+ cells due to PD-L1
  d_HC - chemo induced death of CD4+ cells
  b_HE - (optional) Helper–effector synergy: bidirectional stimulation between CD4⁺ and CD8⁺ subsets

  Breakdown: Immune Cell Influx
  a_I - Intrinsic net change of infused TIL population
  b_IA - Tumor-driven proliferation: TILs expand in response to available antigen targets
  d_IP - Functional suppression: exhaustion or PD-L1–mediated inhibition of infused T cells
  d_IC - damage or depletion of infused TILs due to chemotherapy exposure.
  u_I(t) - external TIL infusion rate or engraftment event (defines treatment timing and dosage)

  Breakdown: Exhaustion/Checkpoint 
  p_H*H_i - patient based baseline flow of PD-L1
  p_A - Tumor-induced upregulation of PD-L1 proportional to tumor burden or antigen presence
  p_C - increase in PD-L1 expression following chemotherapy-related stress
  delta_P - Natural decay or receptor turnover of PD-L1 expression over time.







In [ ]:
CHEMO THERAPY ODES

Parameters:
u_x(t) = concentration of dosage of treatment at time t
lambda_x*X = rate of clearance from the body

Adriamycin
dA/dt = u_a(t) - lamda_a*A 
Cytoxan
dC/dt = u_c(t) - lamda_c*C
Taxol
dT/dt = u_T(t) - lamda_t*t
Tamoxifen
dTam = u_Tam(t) - lamda_tam*Tam




In [ ]:
BREAST CANCER TUMOR ODE

- T(t): tumor burden (cells, mm^3, or normalized units)
- ER(t): ER+ activity (0–1 or normalized score)
- PR(t): PR+ activity (0–1 or normalized score)
- Ki67(t): proliferation index (0–1 or fraction)
- ESR1m(t): ESR1 mutation activity/burden (0–1)
- PI3K(t): PI3K/AKT/mTOR pathway activation (0–1)
- ctDNA(t): circulating tumor DNA burden (e.g., ng/mL or normalized)

inputs for carrying capacity
- BMI(t) or BSA(t)  — either one (normalize to 0–1)
- VEGF(t)           — angiogenesis proxy (use a saturating transform)
- Nutrient(t)       — a combined oxygen+glucose supply index (normalize or saturate)

We build a time-varying carrying capacity K(t) from these signals.

Time-varying carrying capacity
- Normalize each signal to ~[0,1] (z-score or min-max).
- Define:
  K(t) = K0 * [ 1 + w_BMI * BMI_hat(t) + w_VEGF * sat_VEGF(VEGF(t)) + w_N * sat_N(Nutrient(t)) ]

Notes:
- sat_VEGF(z) or sat_N(z) can be z / (K + z) to avoid runaway.

## Modeling style
We use a generalized Lotka–Volterra structure: each variable’s rate = (itself) * (intrinsic rate + interactions) + optional source/sink.
Keep parameters nonnegative where biologically sensible, and clamp fraction-like states (ER, PR, Ki67, ESR1m, PI3K) to [0,1] in code.

---

## ODEs 

### Tumor burden
dotT = T * [
    r_T
  + a_T_Ki67 * Ki67
  + a_T_ER   * ER
  + a_T_PR   * PR
  + a_T_ESR1m* ESR1m
  + a_T_PI3K * PI3K
  - (beta_T * T / K(t))  # logistic crowding with time-varying K(t)
]

### ER activity
dotER = ER * ( r_ER + a_ER_T * T - d_ER ) + c_ER_from_PR * PR * (1 - ER)

### PR activity
dotPR = PR * ( r_PR + a_PR_T * T - d_PR ) + c_PR_from_ER * ER * (1 - PR)

### Ki-67 proliferation index
dotKi67 = Ki67 * ( r_Ki + a_Ki_T * T + a_Ki_PI3K * PI3K - d_Ki )

### ESR1 mutation activity/burden
dotESR1m = ESR1m * ( r_Em + a_Em_T * T - d_Em ) + s_Em_from_ctDNA * ctDNA * (1 - ESR1m)

### PI3K pathway activation
dotPI3K = PI3K * ( r_P + a_P_T * T - d_P ) + a_P_from_ER * ER * (1 - PI3K)

### ctDNA burden
dotctDNA = q_T * T + q_Em * ESR1m * T - delta_ct * ctDNA

---

## What each term means 
- r_*: intrinsic net change when isolated (per-time).
- a_*_*: interaction coefficients (positive increases, negative decreases).
- d_*: decay/turnover toward baseline.
- c_*_from_*: cross-talk between biomarkers; (1 - x) keeps fraction-like variables in [0,1].
- q_T, q_Em: ctDNA production from total tumor and ESR1m-driven resistant clones.
- delta_ct: ctDNA clearance.
- beta_T: strength of logistic self-limitation via K(t).
- K(t): capacity supply from BMI/BSA, VEGF, Nutrient; prevents unrealistic growth.

## Practical notes
- Bounding: clamp ER, PR, Ki67, ESR1m, PI3K to [0,1] each integration step or keep the (1 - x) terms.
- Therapy hooks (optional for later): endocrine therapy can reduce a_T_ER and a_T_PR; PI3K inhibitors reduce a_T_PI3K or increase d_P; chemo can be added as extra death terms on T and as modifiers on biomarkers.


TOXCICITY ODE
- Let D_k(t) be drug exposures (e.g., plasma conc. from PK), one per agent k.
- For each D_k, a one-compartment PK is enough to start:
  dD_k/dt = -k_elim_k * D_k + u_k(t)
  where u_k(t) is the dosing input (bolus or infusion), k_elim_k = ln(2) / t_half_k.

- Use saturating "toxicity drive" from each drug:
  tox_k(D_k) = Emax_k * D_k / (EC50_k + D_k)
  (You can add Hill>1 later if needed.)

- Recovery terms pull biomarkers back toward baseline; damage terms push them away.

------------------------------------------------------------
OPTION A — Single scalar toxicity score (fast/compact)
------------------------------------------------------------
State:
- Xtox(t) in [0, 1] as an overall toxicity burden (0 none, 1 severe).

Dynamics:
- dotXtox = ( sum_k w_k * tox_k(D_k) ) 
            + w_int * Interactions(t)
            - r_rec * Xtox

Notes:
- w_k >= 0 scale each drug's contribution.
- Interactions(t) is optional (e.g., chemo + PD-1 co-toxicity); else set to 0.
- r_rec > 0 is global recovery.
- Clamp Xtox to [0,1] in code.

Pros/cons:
- + Simple, stable, easy to fit when data is sparse.
- – Low clinical interpretability; hard to map to specific adverse events.

----------------------------------------------------------------
OPTION B — 4-biomarker toxicity model (interpretable/reusable)
----------------------------------------------------------------
States (with suggested ranges):
- x_cardiac(t)  in [0,1]  (cardiac damage score; 0 none, 1 severe)
- x_neut(t)     in [0,1]  (normalized neutrophil count; 1 healthy, 0 profoundly low)
- x_immune(t)   in [0,1]  (immune competence; 1 healthy, 0 severely compromised)
- x_neuro(t)    in [0,1]  (neuropathy severity; 0 none, 1 severe)

Drug-specific drives:
- For each drug k, define coefficients on each biomarker: 
  w_k_card, w_k_neut, w_k_imm, w_k_neuro >= 0.
- tox_k(D_k) as above.

Couplings you may want:
- Immune ↔ neutrophils: neutropenia tends to reduce immune competence.
- Neuropathy accumulates slowly and recovers slowly.
- Cardiac injury is often cumulative with slow repair.

Equations (ASCII, per-time units):

1) Cardiac damage score (higher is worse)
dotx_cardiac = ( sum_k w_k_card * tox_k(D_k) ) 
               + w_card_accum * x_cardiac * (1 - x_cardiac)   # cumulative stress
               - r_card_rec * x_cardiac

2) Neutrophil fraction (higher is better)
dotx_neut = - ( sum_k w_k_neut * tox_k(D_k) ) * x_neut        # loss scales with current pool
            + r_neut_rec * (1 - x_neut)                       # marrow recovery towards 1

3) Immune competence (higher is better)
dotx_immune = - ( sum_k w_k_imm * tox_k(D_k) ) * x_immune
              + r_imm_rec * (1 - x_immune)
              + c_neut_to_imm * (x_neut - x_immune)           # neutrophils support immune tone

4) Neuropathy severity (higher is worse)
dotx_neuro = ( sum_k w_k_neuro * tox_k(D_k) )
             + w_neuro_accum * x_neuro * (1 - x_neuro)        # slow accumulation
             - r_neuro_rec * x_neuro

Bounding/implementation:
- After each step, clamp states to valid ranges:
  x_cardiac = min(max(x_cardiac, 0), 1), etc.
- Start with small w_accum terms; set to 0 initially if unstable.

------------------------------------------------------------
Optional aggregate readouts
------------------------------------------------------------
- Overall toxicity index for dashboards:
  Xtox = v_card*x_cardiac + v_neut*(1 - x_neut) 
         + v_imm*(1 - x_immune) + v_neuro*x_neuro
  (Choose v_* to reflect clinical priorities.)
- Grade triggers (pseudo-CTCAE):
  if x_neut < 0.5 -> "Grade ≥3 neutropenia risk"
  if x_cardiac > 0.3 rising -> "Cardiac monitoring"
  if x_neuro > 0.4 -> "Neuropathy dose-hold candidate"

------------------------------------------------------------
Calibration tips
------------------------------------------------------------
- Map x_neut to ANC: x_neut ≈ clamp( ANC / ANC_baseline , 0, 1 ).
- Map x_cardiac to troponin/EF: choose a monotone transform to [0,1].
- x_immune can be composite (lymphocytes, CD4/CD8 ratio, clinical judgement).
- x_neuro can be symptom score or exam-based scale normalized to [0,1].
- Start with one drug and one biomarker to verify signs/magnitudes; then expand.

------------------------------------------------------------
Minimal parameter set (per biomarker)
------------------------------------------------------------
- Recovery: r_card_rec, r_neut_rec, r_imm_rec, r_neuro_rec
- Accumulation (optional): w_card_accum, w_neuro_accum
- Drug weights per biomarker: w_k_card, w_k_neut, w_k_imm, w_k_neuro
- PK: t_half_k (or k_elim_k), EC50_k, Emax_k for each drug k
- Cross-talk: c_neut_to_imm

------------------------------------------------------------
When to choose A vs B
------------------------------------------------------------
- Choose A (single Xtox) if you have only sparse outcomes (e.g., “dose reduced yes/no”).
- Choose B (4D biomarkers) if you have access to labs/signs (ANC, troponin/EF, symptom scores).
- You can always compute Xtox as an aggregate of the 4D model later.
